## Refined Layer

### Carregamento dos datasets da Trusted

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

pd.set_option('display.max_columns', None)

TRUSTED_PATH = '../output/pd/trusted/'
SUFIX = '_trusted.csv'

REFINED_PATH = '../output/pd/refined/'

ds_customers = pd.read_csv(TRUSTED_PATH + 'customers' + SUFIX)
ds_products = pd.read_csv(TRUSTED_PATH + 'products' + SUFIX)
ds_items = pd.read_csv(TRUSTED_PATH + 'items' + SUFIX)
ds_payments = pd.read_csv(TRUSTED_PATH + 'payments' + SUFIX)
ds_reviews = pd.read_csv(TRUSTED_PATH + 'reviews' + SUFIX)
ds_orders = pd.read_csv(TRUSTED_PATH + 'orders' + SUFIX)

#

#### Análise Dataset Orders

Como todos os outros datasets estão confiáveis, chegou a hora de montar nossa tabela `fato_pedido`, porém antes de iniciar a construção da fato precisamos ter as dimensões e dar uma analisada no nosso dataset `Orders`.

In [2]:
ds_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id_int
0,e481f51cbdc54678b7cc49136f2d6af7,68585,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1
1,53cdb2fc8bc7dce0b6741e2150273451,74977,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,2
2,47770eb9100c2d0c44946d9cf07ec65d,555,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3
3,949d5b44dbf5de918fe9c16f97b45f8a,59790,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,4
4,ad21c59c0840e6cb83a9ceb5573f8159,65715,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,5
...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,59296,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,99437
99437,63943bddc261676b46f01ca7ac2f7bd8,76301,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,99438
99438,83c1379a015df1e13d02aae0204711ab,19749,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,99439
99439,11c177c8e97725db2631073c19f07b62,16808,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,99440


In [3]:
ds_orders.drop(columns=["order_id"], inplace=True)
ds_orders.rename(columns={'order_id_int':'order_id'}, inplace=True)
ds_orders['order_purchase_timestamp'] = pd.to_datetime(ds_orders['order_purchase_timestamp'])
ds_orders['order_approved_at'] = pd.to_datetime(ds_orders['order_approved_at'])
ds_orders['order_delivered_customer_date'] = pd.to_datetime(ds_orders['order_delivered_customer_date'])
ds_orders['order_delivered_carrier_date'] = pd.to_datetime(ds_orders['order_delivered_carrier_date'])
ds_orders['order_estimated_delivery_date'] = pd.to_datetime(ds_orders['order_estimated_delivery_date'])
ds_items['shipping_limit_date'] = pd.to_datetime(ds_items['shipping_limit_date'])

In [4]:
ds_orders.isnull().sum()

customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
order_id                            0
dtype: int64

In [5]:
mask = (ds_orders['order_approved_at'].isnull()) & (ds_orders['order_status'] == 'delivered')
ds_orders.loc[mask, 'order_approved_at'] = ds_orders.loc[mask, 'order_purchase_timestamp']

> A primeira alteração que eu resolvi fazer foi adicionar uma data de aprovação dos pedidos aos pedidos que satisfazessem duas condições: Fossem nulos e já estivessem sido entregues. Pois não faz sentido um pedido não ter sido aprovado porém entregue, então defini como data de aprovação do pedido o instante em que foi solicitada a compra.

In [6]:
ds_orders.isnull().sum()

customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 146
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
order_id                            0
dtype: int64

# 

> Abaixo, ocorrerá uma série de atualizações no nosso dataset. Resumidamente, estou padronizando os valores nulos com valores padrão para evitar erros na análise ou no carregamento das dimensões para o nosso DataWarehouse

In [7]:
# Para os pedidos que ainda não foram aprovados atualizei os datas
# Para o valor mínimo no timestamp

mask = (ds_orders['order_approved_at'].isnull())

ds_orders.loc[mask, 'order_approved_at'] = pd.Timestamp('1900-12-31')
ds_orders.loc[mask, 'order_delivered_carrier_date'] = pd.Timestamp('1900-12-31')
ds_orders.loc[mask, 'order_delivered_customer_date'] = pd.Timestamp('1900-12-31')

In [8]:
# Para os pedidos que foram entregues, porém sem aprovação, atualizei o status das datas
# Para o valor mínimo no timestamp

mask = (ds_orders['order_delivered_carrier_date'].isnull()) & (ds_orders.order_approved_at != pd.NaT)

ds_orders.loc[mask, 'order_delivered_carrier_date'] = pd.Timestamp('1900-12-31')
ds_orders.loc[mask, 'order_delivered_customer_date'] = pd.Timestamp('1900-12-31')

In [9]:
# Atualizando os pedidos que não possuem uma data de entrega do cliente (ainda)  atualizei os datas
# Para o valor mínimo no timestamp

mask = (ds_orders['order_delivered_customer_date'].isnull())

ds_orders.loc[mask, 'order_delivered_customer_date'] = pd.Timestamp('1900-12-31')

In [10]:
ds_orders.isnull().sum()

customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
order_id                         0
dtype: int64

In [11]:
ds_orders = ds_orders.convert_dtypes()

In [12]:
ds_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   customer_id                    99441 non-null  Int64         
 1   order_status                   99441 non-null  string        
 2   order_purchase_timestamp       99441 non-null  datetime64[ns]
 3   order_approved_at              99441 non-null  datetime64[ns]
 4   order_delivered_carrier_date   99441 non-null  datetime64[ns]
 5   order_delivered_customer_date  99441 non-null  datetime64[ns]
 6   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 7   order_id                       99441 non-null  Int64         
dtypes: Int64(2), datetime64[ns](5), string(1)
memory usage: 6.3 MB


#

#### Construção da Dimensão Tempo Data e Tempo Instante

Aqui vai uma solução de análise temporal que aumentará a quantidade de campos porém reduzirá a complexidade de análise. Separei em duas dimensões de tempo, uma representando, literalmente o instante e outra representando a data.

In [13]:
# Garantindo que seja um datetime, pois não fiz a conversão anteriormente

ds_reviews['review_answer_timestamp'] = pd.to_datetime(ds_reviews['review_answer_timestamp'])
ds_reviews['review_creation_date'] = pd.to_datetime(ds_reviews['review_creation_date'])

dim_tempo_instante = pd.DataFrame({
    'tempo_instante': pd.concat([
        ds_orders['order_purchase_timestamp'],
        ds_orders['order_approved_at'],
        ds_orders['order_delivered_carrier_date'],
        ds_orders['order_delivered_customer_date'],
        ds_orders['order_estimated_delivery_date'],
        ds_reviews['review_answer_timestamp'],
        ds_reviews['review_creation_date'],
        ds_items['shipping_limit_date']
    ]).unique()
})

dim_tempo_instante['hora'] = dim_tempo_instante['tempo_instante'].dt.hour
dim_tempo_instante['minuto'] = dim_tempo_instante['tempo_instante'].dt.minute
dim_tempo_instante['segundo'] = dim_tempo_instante['tempo_instante'].dt.second

dim_tempo_instante.head(2)

,tempo_instante,hora,minuto,segundo
0,2017-10-02 10:56:33,10,56,33
1,2018-07-24 20:41:37,20,41,37


In [14]:
dim_tempo_data = pd.DataFrame({
    'tempo_data': pd.concat([
        ds_orders['order_purchase_timestamp'].dt.date,
        ds_orders['order_approved_at'].dt.date,
        ds_orders['order_delivered_carrier_date'].dt.date,
        ds_orders['order_delivered_customer_date'].dt.date,
        ds_orders['order_estimated_delivery_date'].dt.date,
        ds_reviews['review_creation_date'].dt.date,
        ds_reviews['review_answer_timestamp'].dt.date,
        ds_items['shipping_limit_date'].dt.date
    ]).unique()
})

dim_tempo_data['dia'] = pd.to_datetime(dim_tempo_data['tempo_data']).dt.day
dim_tempo_data['mes'] = pd.to_datetime(dim_tempo_data['tempo_data']).dt.month
dim_tempo_data['ano'] = pd.to_datetime(dim_tempo_data['tempo_data']).dt.year

dim_tempo_data.head(2)

,tempo_data,dia,mes,ano
0,2017-10-02,2,10,2017
1,2018-07-24,24,7,2018


In [15]:
dim_tempo_instante.reset_index(inplace=True)
dim_tempo_instante.rename(columns={'index': 'tempo_instante_id'}, inplace=True)
dim_tempo_instante.drop(columns=['tempo_instante'], inplace=True)
dim_tempo_instante = dim_tempo_instante.drop_duplicates(subset=['hora', 'minuto', 'segundo'])

dim_tempo_data.reset_index(inplace=True)
dim_tempo_data.rename(columns={'index': 'tempo_data_id'}, inplace=True)
dim_tempo_data.drop(columns=['tempo_data'], inplace=True)

In [16]:
dim_tempo_data.head(2)

,tempo_data_id,dia,mes,ano
0,0,2,10,2017
1,1,24,7,2018


In [17]:
dim_tempo_instante.head(2)

,tempo_instante_id,hora,minuto,segundo
0,0,10,56,33
1,1,20,41,37


In [18]:
dim_tempo_instante.to_csv(REFINED_PATH + 'dim_tempo_instante.csv', index=False)
dim_tempo_data.to_csv(REFINED_PATH + 'dim_tempo_data.csv', index=False)

#

#### Construção da Dimensão Clientes

In [19]:
ds_customers

,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,1,14409,franca,SP
1,2,9790,sao bernardo do campo,SP
2,3,1151,sao paulo,SP
3,4,8775,mogi das cruzes,SP
4,5,13056,campinas,SP
...,...,...,...,...
96091,96092,3937,sao paulo,SP
96092,96093,6764,taboao da serra,SP
96093,96094,60115,fortaleza,CE
96094,96095,92120,canoas,RS


In [20]:
dim_cliente = ds_customers.copy()
dim_cliente.drop_duplicates(subset=['customer_id'], keep='first', inplace=True)
dim_cliente.rename(columns={'customer_id': 'cliente_id', 'customer_zip_code_prefix':'cliente_codigo_postal', 
                                                    'customer_city':'cliente_cidade', 'customer_state':'cliente_estado'}, inplace=True)
dim_cliente.to_csv(REFINED_PATH + 'dim_cliente.csv', index=False)

In [21]:
dim_cliente.head(5)

,cliente_id,cliente_codigo_postal,cliente_cidade,cliente_estado
0,1,14409,franca,SP
1,2,9790,sao bernardo do campo,SP
2,3,1151,sao paulo,SP
3,4,8775,mogi das cruzes,SP
4,5,13056,campinas,SP


#

#### Construção da Dimensão Pagamentos

In [22]:
ds_payments.head(2)

,order_id,payment_sequential,payment_type,payment_installments,payment_value,payment_id
0,75269,1,credit_card,8,99.33,1
1,98161,1,credit_card,1,24.39,2


In [23]:
dim_pagamento = ds_payments[['payment_id', 'payment_type']].copy()
dim_pagamento.rename(columns={'payment_id':'pagamento_id', 'payment_type':'pagamento_tipo'},inplace=True)

In [24]:
# Removendo as duplicatas para termos apenas os tipos de pagamento nesta dimensão

dim_pagamento.drop_duplicates(subset=['pagamento_tipo'], keep='first', inplace=True)
dim_pagamento['pagamento_id'] = range(1, len(dim_pagamento) + 1)

In [25]:
dim_pagamento

,pagamento_id,pagamento_tipo
0,1,credit_card
9,2,boleto
25,3,voucher
85,4,debit_card
51280,5,not_defined


In [26]:
dim_pagamento.to_csv(REFINED_PATH + 'dim_pagamento.csv', index=False)

#

In [27]:
# Atualizando o ds_payment para facilitar o merge a tabela fato

ds_payments = pd.merge(ds_payments, dim_pagamento, left_on='payment_type', right_on='pagamento_tipo')
ds_payments.drop(columns=['payment_id', 'payment_type', 'pagamento_tipo'], inplace=True)
ds_payments.rename(columns={'payment_sequential':'pagamento_formas_distintas',
                           'payment_installments':'pagamento_parcelamentos',
                           'payment_value':'pagamento_valor'}, inplace=True)
ds_payments.head(2)

,order_id,pagamento_formas_distintas,pagamento_parcelamentos,pagamento_valor,pagamento_id
0,75269,1,8,99.33,1
1,98161,1,1,24.39,1


#

#### Construção dimensão Review

In [28]:
ds_reviews = ds_reviews.convert_dtypes()
ds_reviews['review_answer_timestamp'] = pd.to_datetime(ds_reviews['review_answer_timestamp'])
ds_reviews['review_creation_date'] = pd.to_datetime(ds_reviews['review_creation_date'])

In [29]:
ds_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98673 entries, 0 to 98672
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   order_id                 98673 non-null  Int64         
 1   review_score             98673 non-null  Int64         
 2   review_comment_title     98673 non-null  string        
 3   review_comment_message   98673 non-null  string        
 4   review_creation_date     98673 non-null  datetime64[ns]
 5   review_answer_timestamp  98673 non-null  datetime64[ns]
 6   review_id                98673 non-null  Int64         
dtypes: Int64(3), datetime64[ns](2), string(2)
memory usage: 5.6 MB


In [30]:
ds_reviews.head(2)

,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,review_id
0,1872,4,Sem Titulo,Sem Comentários,2018-01-18,2018-01-18 21:46:59,1
1,90489,5,Sem Titulo,Sem Comentários,2018-03-10,2018-03-11 03:05:13,2


In [31]:
# Script que montei pra extrair a data e tirar dela o id referente a dimensão data e instante

for field in ['review_creation_date', 'review_answer_timestamp']:
    ds_reviews[field + '_dia'] = ds_reviews[field].dt.day
    ds_reviews[field + '_mes'] = ds_reviews[field].dt.month
    ds_reviews[field + '_ano'] = ds_reviews[field].dt.year
    ds_reviews[field + '_hora'] = ds_reviews[field].dt.hour
    ds_reviews[field + '_minuto'] = ds_reviews[field].dt.minute
    ds_reviews[field + '_segundo'] = ds_reviews[field].dt.second

def obter_ids(dataframe, campo_base):

    # Data
    dataframe = pd.merge(ds_reviews, dim_tempo_data, how='left',
                         left_on=[campo_base + '_ano', campo_base + '_mes', campo_base + '_dia'],
                         right_on=['ano', 'mes', 'dia'])
    dataframe = dataframe.rename(columns={'tempo_data_id': campo_base + '_data_id'})
    dataframe.drop(columns=['ano', 'mes', 'dia'], inplace=True)

    # Instante
    dataframe = pd.merge(dataframe, dim_tempo_instante, how='left', 
                         left_on=[campo_base + '_hora', campo_base + '_minuto', campo_base + '_segundo'],
                         right_on=['hora', 'minuto', 'segundo'])
    dataframe = dataframe.rename(columns={'tempo_instante_id': campo_base + '_instante_id'})
    dataframe.drop(columns=['hora', 'minuto', 'segundo'], inplace=True)

    return dataframe

for field in ['review_creation_date', 'review_answer_timestamp']:
    ds_reviews = obter_ids(ds_reviews, field)

colunas_aux = []
for field in ['review_creation_date', 'review_answer_timestamp']:
    colunas_aux.extend([field + '_ano', field + '_mes', field + '_dia', field + '_hora', field + '_minuto', field + '_segundo'])

ds_reviews.drop(columns=colunas_aux, inplace=True)

In [32]:
ds_reviews.head(2)

,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,review_id,review_creation_date_data_id,review_creation_date_instante_id,review_answer_timestamp_data_id,review_answer_timestamp_instante_id
0,1872,4,Sem Titulo,Sem Comentários,2018-01-18,2018-01-18 21:46:59,1,152,94517,152,3580
1,90489,5,Sem Titulo,Sem Comentários,2018-03-10,2018-03-11 03:05:13,2,357,94517,449,100781


In [33]:
ds_reviews.isnull().sum()

order_id                               0
review_score                           0
review_comment_title                   0
review_comment_message                 0
review_creation_date                   0
review_answer_timestamp                0
review_id                              0
review_creation_date_data_id           0
review_creation_date_instante_id       0
review_answer_timestamp_data_id        0
review_answer_timestamp_instante_id    0
dtype: int64

In [34]:
ds_reviews.drop(columns=['review_creation_date', 'review_answer_timestamp'], inplace=True)

ds_review = ds_reviews.rename(columns={
    'order_id': 'pedido_id',
    'review_score':'avaliacao_pontuacao',
    'review_id': 'avaliacao_id',
    'review_comment_title': 'avaliacao_titulo_comentario',
    'review_comment_message': 'avaliacao_mensagem_comentario',
    'review_creation_date_data_id': 'avaliacao_data_criacao_id',
    'review_creation_date_instante_id': 'avaliacao_instante_criacao_id',
    'review_answer_timestamp_data_id': 'avaliacao_data_resposta_id',
    'review_answer_timestamp_instante_id': 'avaliacao_instante_resposta_id'
})

In [35]:
dim_avaliacao = ds_review.copy()

dim_avaliacao.drop(columns=['pedido_id', 'avaliacao_pontuacao', 
                   'avaliacao_data_criacao_id', 'avaliacao_instante_criacao_id',
                  'avaliacao_data_resposta_id', 'avaliacao_instante_resposta_id'], inplace=True)
dim_avaliacao.drop_duplicates(subset=['avaliacao_titulo_comentario', 'avaliacao_mensagem_comentario'], inplace=True)

In [36]:
dim_avaliacao['avaliacao_id'] = range(1, len(dim_avaliacao) + 1)
dim_avaliacao.head(2)

,avaliacao_titulo_comentario,avaliacao_mensagem_comentario,avaliacao_id
0,Sem Titulo,Sem Comentários,1
3,Sem Titulo,Recebi bem antes do prazo estipulado.,2


In [37]:
ds_reviews.head(2)

,order_id,review_score,review_comment_title,review_comment_message,review_id,review_creation_date_data_id,review_creation_date_instante_id,review_answer_timestamp_data_id,review_answer_timestamp_instante_id
0,1872,4,Sem Titulo,Sem Comentários,1,152,94517,152,3580
1,90489,5,Sem Titulo,Sem Comentários,2,357,94517,449,100781


In [38]:
ds_reviews = pd.merge(ds_reviews, dim_avaliacao, left_on=['review_comment_title', 'review_comment_message'], right_on=['avaliacao_titulo_comentario', 'avaliacao_mensagem_comentario'])

In [39]:
ds_reviews.drop(columns=['review_comment_title', 'review_comment_message', 'review_id', 'review_answer_timestamp_data_id'], inplace=True)
ds_reviews.rename(columns={'review_score': 'avaliacao_pontuacao', 'review_creation_date_data_id':'avaliacao_criacao_data_id'
                          ,'review_creation_date_instante_id':'avaliacao_criacao_instante_id',
                          'review_answer_timestamp_instante_id':'avaliacao_resposta_instante_id'}, inplace=True)

In [40]:
ds_reviews.head(2)

,order_id,avaliacao_pontuacao,avaliacao_criacao_data_id,avaliacao_criacao_instante_id,avaliacao_resposta_instante_id,avaliacao_titulo_comentario,avaliacao_mensagem_comentario,avaliacao_id
0,1872,4,152,94517,3580,Sem Titulo,Sem Comentários,1
1,90489,5,357,94517,100781,Sem Titulo,Sem Comentários,1


In [41]:
dim_avaliacao.to_csv(REFINED_PATH + 'dim_avaliacao.csv', index=False)

#

#### Construção dimensão Pedido

In [42]:
ds_orders.head(2)

,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id
0,68585,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1
1,74977,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,2


In [43]:
ds_orders[['order_id', 'order_status']].head(2)

,order_id,order_status
0,1,delivered
1,2,delivered


In [44]:
dim_pedido = ds_orders[['order_id', 'order_status']].copy()
dim_pedido.rename(columns={'order_id':'pedido_id', 'order_status':'pedido_status'}, inplace=True)
dim_pedido.drop_duplicates(subset=['pedido_status'], keep='first', inplace=True)
dim_pedido['pedido_id'] = range(1, len(dim_pedido) + 1)

In [45]:
dim_pedido

,pedido_id,pedido_status
0,1,delivered
6,2,invoiced
44,3,shipped
128,4,processing
266,5,unavailable
397,6,canceled
7434,7,created
44897,8,approved


In [46]:
ds_orders.head(2)

,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id
0,68585,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1
1,74977,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,2


In [47]:
for field in ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']:
    ds_orders[field + '_dia'] = ds_orders[field].dt.day
    ds_orders[field + '_mes'] = ds_orders[field].dt.month
    ds_orders[field + '_ano'] = ds_orders[field].dt.year
    ds_orders[field + '_hora'] = ds_orders[field].dt.hour
    ds_orders[field + '_minuto'] = ds_orders[field].dt.minute
    ds_orders[field + '_segundo'] = ds_orders[field].dt.second

def obter_ids(dataframe, campo_base):
    dataframe = pd.merge(ds_orders, dim_tempo_data, how='left',
                         left_on=[campo_base + '_ano', campo_base + '_mes', campo_base + '_dia'],
                         right_on=['ano', 'mes', 'dia'])

    dataframe = dataframe.rename(columns={'tempo_data_id': campo_base + '_data_id'})
    
    dataframe.drop(columns=['ano', 'mes', 'dia'], inplace=True)
    
    dataframe = pd.merge(dataframe, dim_tempo_instante, how='left', 
                         left_on=[campo_base + '_hora', campo_base + '_minuto', campo_base + '_segundo'],
                         right_on=['hora', 'minuto', 'segundo'])
    dataframe = dataframe.rename(columns={'tempo_instante_id': campo_base + '_instante_id'})
    
    dataframe.drop(columns=['hora', 'minuto', 'segundo'], inplace=True)

    return dataframe

for field in ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']:
    ds_orders = obter_ids(ds_orders, field)

colunas_aux = []
for field in ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']:
    colunas_aux.extend([field + '_ano', field + '_mes', field + '_dia', field + '_hora', field + '_minuto', field + '_segundo'])
colunas_aux.extend(['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date'])

ds_orders.drop(columns=colunas_aux, inplace=True)

In [48]:
ds_orders.columns

Index(['customer_id', 'order_status', 'order_estimated_delivery_date',
       'order_id', 'order_purchase_timestamp_data_id',
       'order_purchase_timestamp_instante_id', 'order_approved_at_data_id',
       'order_approved_at_instante_id', 'order_delivered_carrier_date_data_id',
       'order_delivered_carrier_date_instante_id',
       'order_delivered_customer_date_data_id',
       'order_delivered_customer_date_instante_id'],
      dtype='object')

In [49]:
ds_orders['order_estimated_delivery_date_dia'] = ds_orders['order_estimated_delivery_date'].dt.day
ds_orders['order_estimated_delivery_date_mes'] = ds_orders['order_estimated_delivery_date'].dt.month
ds_orders['order_estimated_delivery_date_ano'] = ds_orders['order_estimated_delivery_date'].dt.year

ds_orders = pd.merge(ds_orders, dim_tempo_data, how='left',
                         left_on=['order_estimated_delivery_date_ano', 'order_estimated_delivery_date_mes', 'order_estimated_delivery_date_dia'],
                         right_on=['ano', 'mes', 'dia'])

ds_orders.drop(columns=['ano', 'mes', 'dia', 
                        'order_estimated_delivery_date', 
                        'order_estimated_delivery_date_dia', 
                        'order_estimated_delivery_date_mes', 
                        'order_estimated_delivery_date_ano'], inplace=True)

ds_orders.rename(columns={'tempo_data_id': 'pedido_entrega_estimada_data_id'}, inplace=True)

In [50]:
ds_orders.columns

Index(['customer_id', 'order_status', 'order_id',
       'order_purchase_timestamp_data_id',
       'order_purchase_timestamp_instante_id', 'order_approved_at_data_id',
       'order_approved_at_instante_id', 'order_delivered_carrier_date_data_id',
       'order_delivered_carrier_date_instante_id',
       'order_delivered_customer_date_data_id',
       'order_delivered_customer_date_instante_id',
       'pedido_entrega_estimada_data_id'],
      dtype='object')

In [51]:
dim_pedido.head(2)

,pedido_id,pedido_status
0,1,delivered
6,2,invoiced


In [52]:
ds_orders = pd.merge(ds_orders, dim_pedido, left_on='order_status', right_on='pedido_status')
ds_orders.rename(columns={
    'order_purchase_timestamp_data_id':'pedido_compra_data_id',
    'order_purchase_timestamp_instante_id':'pedido_compra_instante_id',
    'order_approved_at_data_id':'pedido_aprovacao_data_id',
    'order_approved_at_instante_id':'pedido_aprovacao_instante_id',
    'order_delivered_carrier_date_data_id':'pedido_entregue_operador_data_id',
    'order_delivered_carrier_date_instante_id':'pedido_entregue_operador_instante_id',
    'order_delivered_customer_date_data_id':'pedido_entregue_cliente_data_id',
    'order_delivered_customer_date_instante_id':'pedido_entregue_cliente_instante_id',
    'customer_id':'cliente_id',
}, inplace=True)

ds_orders.drop(columns='order_status', inplace=True)

In [53]:
ds_orders.head(2)

,cliente_id,order_id,pedido_compra_data_id,pedido_compra_instante_id,pedido_aprovacao_data_id,pedido_aprovacao_instante_id,pedido_entregue_operador_data_id,pedido_entregue_operador_instante_id,pedido_entregue_cliente_data_id,pedido_entregue_cliente_instante_id,pedido_entrega_estimada_data_id,pedido_id,pedido_status
0,68585,1,0,0,0,45847,138,54647,588,43996,273,1,delivered
1,74977,2,1,1,47,98876,47,2938,359,80412,474,1,delivered


In [54]:
dim_pedido.to_csv(REFINED_PATH + 'dim_pedido.csv', index=False)

#

#### Construção dimensão Produto

In [55]:
ds_products.columns

Index(['product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_id'],
      dtype='object')

In [56]:
dim_produto = ds_products.rename(columns={'product_id':'produto_id',
                                          'product_category_name':'produto_categoria', 
                                          'product_name_lenght':'produto_tamanho_nome',
                                          'product_description_lenght':'produto_tamanho_descricao', 
                                          'product_photos_qty':'produto_qtd_fotos', 
                                          'product_weight_g':'produto_peso_g',
                                          'product_length_cm':'produto_largura_cm', 
                                          'product_height_cm':'produto_altura_cm', 
                                          'product_width_cm':'produto_comprimento_cm'
                                          }).copy()

dim_produto.drop(columns=['produto_tamanho_nome', 'produto_tamanho_descricao'], inplace=True)

In [57]:
dim_produto.to_csv(REFINED_PATH + 'dim_produto.csv', index=False)

#

#### Construção Tabela Fato

In [58]:
ds_items.head(2)

,order_id,product_id,shipping_limit_date,price,freight_value,quantidade
0,85268,25864,2017-09-19 09:45:35,58.9,13.29,1
1,71854,27229,2017-05-03 11:05:13,239.9,19.93,1


In [59]:
ds_items['shipping_limit_date'] = pd.to_datetime(ds_items['shipping_limit_date'])

ds_items['shipping_limit_date_dia'] = ds_items['shipping_limit_date'].dt.day
ds_items['shipping_limit_date_mes'] = ds_items['shipping_limit_date'].dt.month
ds_items['shipping_limit_date_ano'] = ds_items['shipping_limit_date'].dt.year
ds_items['shipping_limit_date_hora'] = ds_items['shipping_limit_date'].dt.hour
ds_items['shipping_limit_date_minuto'] = ds_items['shipping_limit_date'].dt.minute
ds_items['shipping_limit_date_segundo'] = ds_items['shipping_limit_date'].dt.second

# Data

ds_items = pd.merge(ds_items, dim_tempo_data, how='left',
                         left_on=['shipping_limit_date_ano', 'shipping_limit_date_mes', 'shipping_limit_date_dia'],
                         right_on=['ano', 'mes', 'dia'])

ds_items.drop(columns=['ano', 'mes', 'dia', 
                        'shipping_limit_date', 
                        'shipping_limit_date_dia', 
                        'shipping_limit_date_mes', 
                        'shipping_limit_date_ano'], inplace=True)

# Instante

ds_items = pd.merge(ds_items, dim_tempo_instante, how='left',
                         left_on=['shipping_limit_date_hora', 'shipping_limit_date_minuto', 'shipping_limit_date_segundo'],
                         right_on=['hora', 'minuto', 'segundo'])

ds_items.drop(columns=['hora', 'minuto', 'segundo', 
                        'shipping_limit_date_hora', 
                        'shipping_limit_date_minuto', 
                        'shipping_limit_date_segundo'], inplace=True)


ds_items.rename(columns={'tempo_data_id': 'envio_limite_data_id', 
                         'tempo_instante_id':'envio_limite_instante_id',
                        'price':'preco',
                        'freight_value':'frete_valor'}, inplace=True)

ds_items.head(2)

,order_id,product_id,preco,frete_valor,quantidade,envio_limite_data_id,envio_limite_instante_id
0,85268,25864,58.9,13.29,1,83,39753
1,71854,27229,239.9,19.93,1,309,82240


In [60]:
ds_orders.rename(columns={'order_id_int': 'order_id'}, inplace=True)

In [61]:
ds_items.head(2)

,order_id,product_id,preco,frete_valor,quantidade,envio_limite_data_id,envio_limite_instante_id
0,85268,25864,58.9,13.29,1,83,39753
1,71854,27229,239.9,19.93,1,309,82240


In [62]:
ds_customers.head(2)

,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,1,14409,franca,SP
1,2,9790,sao bernardo do campo,SP


In [63]:
ds_payments.head(2)

,order_id,pagamento_formas_distintas,pagamento_parcelamentos,pagamento_valor,pagamento_id
0,75269,1,8,99.33,1
1,98161,1,1,24.39,1


In [64]:
ds_reviews.head(2)

,order_id,avaliacao_pontuacao,avaliacao_criacao_data_id,avaliacao_criacao_instante_id,avaliacao_resposta_instante_id,avaliacao_titulo_comentario,avaliacao_mensagem_comentario,avaliacao_id
0,1872,4,152,94517,3580,Sem Titulo,Sem Comentários,1
1,90489,5,357,94517,100781,Sem Titulo,Sem Comentários,1


In [65]:
ds_orders

,cliente_id,order_id,pedido_compra_data_id,pedido_compra_instante_id,pedido_aprovacao_data_id,pedido_aprovacao_instante_id,pedido_entregue_operador_data_id,pedido_entregue_operador_instante_id,pedido_entregue_cliente_data_id,pedido_entregue_cliente_instante_id,pedido_entrega_estimada_data_id,pedido_id,pedido_status
0,68585,1,0,0,0,45847,138,54647,588,43996,273,1,delivered
1,74977,2,1,1,47,98876,47,2938,359,80412,474,1,delivered
2,555,3,2,2,2,98877,2,19959,447,226702,663,1,delivered
3,59790,4,3,3,3,29539,88,73761,116,58085,48,1,delivered
4,65715,5,4,4,4,60290,131,122361,350,43565,457,1,delivered
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,59296,99437,164,61253,164,61253,351,46900,591,3325,119,1,delivered
99437,76301,99438,93,98871,93,61386,430,129012,150,28154,444,1,delivered
99438,19749,99439,479,94837,479,67456,274,125207,348,48,97,1,delivered
99439,16808,99440,21,79990,21,188050,213,52130,191,32935,66,1,delivered


In [66]:
fato_pedido = pd.merge(ds_orders, ds_customers['customer_id'], right_on='customer_id', left_on='cliente_id')
fato_pedido = pd.merge(fato_pedido, ds_payments, on='order_id')
fato_pedido = pd.merge(fato_pedido, ds_items, on='order_id')
fato_pedido = pd.merge(fato_pedido, ds_reviews, on='order_id')

In [67]:
fato_pedido.head(2)

,cliente_id,order_id,pedido_compra_data_id,pedido_compra_instante_id,pedido_aprovacao_data_id,pedido_aprovacao_instante_id,pedido_entregue_operador_data_id,pedido_entregue_operador_instante_id,pedido_entregue_cliente_data_id,pedido_entregue_cliente_instante_id,pedido_entrega_estimada_data_id,pedido_id,pedido_status,customer_id,pagamento_formas_distintas,pagamento_parcelamentos,pagamento_valor,pagamento_id,product_id,preco,frete_valor,quantidade,envio_limite_data_id,envio_limite_instante_id,avaliacao_pontuacao,avaliacao_criacao_data_id,avaliacao_criacao_instante_id,avaliacao_resposta_instante_id,avaliacao_titulo_comentario,avaliacao_mensagem_comentario,avaliacao_id
0,68585,1,0,0,0,45847,138,54647,588,43996,273,1,delivered,68585,1,1,18.12,1,2350,29.99,8.72,1,282,45847,4,246,94517,162276,Sem Titulo,"Não testei o produto ainda, mas ele veio corre...",13034
1,68585,1,0,0,0,45847,138,54647,588,43996,273,1,delivered,68585,3,1,2.00,3,2350,29.99,8.72,1,282,45847,4,246,94517,162276,Sem Titulo,"Não testei o produto ainda, mas ele veio corre...",13034


In [68]:
fato_pedido.drop(columns=['avaliacao_titulo_comentario','order_id','customer_id','avaliacao_mensagem_comentario', 'pedido_status'], inplace=True)
fato_pedido.rename(columns={'product_id': 'produto_id', 'preco':'preco_unitario', 'qtd':'quantidade'}, inplace=True)

In [69]:
fato_pedido.head(10)

,cliente_id,pedido_compra_data_id,pedido_compra_instante_id,pedido_aprovacao_data_id,pedido_aprovacao_instante_id,pedido_entregue_operador_data_id,pedido_entregue_operador_instante_id,pedido_entregue_cliente_data_id,pedido_entregue_cliente_instante_id,pedido_entrega_estimada_data_id,pedido_id,pagamento_formas_distintas,pagamento_parcelamentos,pagamento_valor,pagamento_id,produto_id,preco_unitario,frete_valor,quantidade,envio_limite_data_id,envio_limite_instante_id,avaliacao_pontuacao,avaliacao_criacao_data_id,avaliacao_criacao_instante_id,avaliacao_resposta_instante_id,avaliacao_id
0,68585,0,0,0,45847,138,54647,588,43996,273,1,1,1,18.12,1,2350,29.99,8.72,1,282,45847,4,246,94517,162276,13034
1,68585,0,0,0,45847,138,54647,588,43996,273,1,3,1,2.00,3,2350,29.99,8.72,1,282,45847,4,246,94517,162276,13034
2,68585,0,0,0,45847,138,54647,588,43996,273,1,2,1,18.59,3,2350,29.99,8.72,1,282,45847,4,246,94517,162276,13034
3,74977,1,1,47,98876,47,2938,359,80412,474,1,1,1,141.46,2,20549,118.70,22.76,1,92,98876,4,2,94517,205618,11640
4,555,2,2,2,98877,2,19959,447,226702,663,1,1,3,179.12,1,12228,159.90,19.22,1,474,98877,5,554,94517,21715,1
5,59790,3,3,3,29539,88,73761,116,58085,48,1,1,1,72.20,1,29924,45.00,27.20,1,368,29539,5,312,94517,38545,29826
6,65715,4,4,4,60290,131,122361,350,43565,457,1,1,1,28.62,1,11900,19.90,8.72,1,57,3990,5,32,94517,55485,1
7,66777,5,5,5,487,420,188057,410,13652,162,1,1,6,175.26,1,18705,147.90,27.36,1,10,487,4,344,94517,11389,1
8,74070,6,6,307,98881,634,94517,634,94517,75,2,1,1,65.95,1,23157,49.90,16.05,1,238,98881,2,498,94517,66235,11455
9,84351,7,7,7,95325,461,188058,236,83160,503,1,1,3,75.16,1,9304,59.99,15.17,1,461,95325,5,473,94517,367609,1


In [70]:
fato_pedido = fato_pedido.convert_dtypes()

In [71]:
fato_pedido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106175 entries, 0 to 106174
Data columns (total 26 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   cliente_id                            106175 non-null  Int64  
 1   pedido_compra_data_id                 106175 non-null  Int64  
 2   pedido_compra_instante_id             106175 non-null  Int64  
 3   pedido_aprovacao_data_id              106175 non-null  Int64  
 4   pedido_aprovacao_instante_id          106175 non-null  Int64  
 5   pedido_entregue_operador_data_id      106175 non-null  Int64  
 6   pedido_entregue_operador_instante_id  106175 non-null  Int64  
 7   pedido_entregue_cliente_data_id       106175 non-null  Int64  
 8   pedido_entregue_cliente_instante_id   106175 non-null  Int64  
 9   pedido_entrega_estimada_data_id       106175 non-null  Int64  
 10  pedido_id                             106175 non-null  Int64  
 11  

In [72]:
fato_pedido.isnull().sum()

cliente_id                              0
pedido_compra_data_id                   0
pedido_compra_instante_id               0
pedido_aprovacao_data_id                0
pedido_aprovacao_instante_id            0
pedido_entregue_operador_data_id        0
pedido_entregue_operador_instante_id    0
pedido_entregue_cliente_data_id         0
pedido_entregue_cliente_instante_id     0
pedido_entrega_estimada_data_id         0
pedido_id                               0
pagamento_formas_distintas              0
pagamento_parcelamentos                 0
pagamento_valor                         0
pagamento_id                            0
produto_id                              0
preco_unitario                          0
frete_valor                             0
quantidade                              0
envio_limite_data_id                    0
envio_limite_instante_id                0
avaliacao_pontuacao                     0
avaliacao_criacao_data_id               0
avaliacao_criacao_instante_id     

In [74]:
fato_pedido['fato_pedido_id'] = range(1, len(fato_pedido) + 1)

In [75]:
fato_pedido.to_csv(REFINED_PATH + 'fato_pedido.csv', index=False)

#

#

#